In [ ]:
# import cell

import numpy as np

## tensordot

[numpy.tensordot](https://numpy.org/doc/stable/reference/generated/numpy.tensordot.html)

**tensor** is just a matrix with more than 2 dimensions.

`tensordot(a, b, axes=2)`: Compute tensor dot product *along* specified axes.

Given two tensors, `a` and `b`, and an array_like object containing two array_like objects, ``(a_axes, b_axes)``, <u>sum the products</u> of `a`'s and `b`'s elements (components) over the axes specified by ``a_axes`` and ``b_axes``.  
The third argument can be a single non-negative integer_like scalar, ``N``; if it is such, then the *last* ``N`` dimensions of `a` and the *first* ``N`` dimensions of `b` are summed over.  

`axes`: int or (2,) array_like:

- integer_like If an int N, sum over the **last** N axes of a and the **first** N axes of b in order. The sizes of the corresponding axes must match.
- (2,) array_like Or, a list of axes to be summed over, first sequence applying to a, second to b. Both elements array_like must be of the same length.

Three common use cases are:

- `axes = 0` : tensor product: $a\otimes b$
- `axes = 1` : tensor dot product: $a\cdot b$
- `axes = 2` : (default) tensor double contraction: $a:b$

When axes is integer_like, the sequence for evaluation will be: **first** the -Nth axis in a and 0th axis in b, and the -1th axis in a and Nth axis in b **last**.

当 axes 为整数时，默认为2（至少有两条轴），一般针对二维及以上的数组。

- 指定沿着数组a的后n个轴和数组b的前n个轴分别进行点积，即对应位置元素相乘，再整体求和。

当 axes 为 array_like 时，可以为tuple，也可为list，也可以为 tuple(tuple) 或 tuple(list)，指明 summed over 的方向。

[numpy中的tensordot](https://www.cnblogs.com/traditional/p/12639487.html)  
[Understanding tensordot](https://stackoverflow.com/questions/41870228/understanding-tensordot)  
[Tensordot — Multidimensional Dot Product — Explained](https://medium.com/analytics-vidhya/tensordot-explained-6673cfa5697f)  


### 1D vector

The visual below shows how two vectors of length 3 transform into a 3x3 matrix.

`axes = 0`: last 0 of x is **scalar**, and fist 0 of y is the whole row(vector).

```Python
# A single instance of x in this case is each scalar value of x, 
# and a single instance of y is the whole y matrix.
# x的3个标量变成3个向量
np.dot(x[0], y)
np.dot(x[1], y)
np.dot(x[2], y)
```

![vector-tensordot-0](https://miro.medium.com/max/1400/1*PhZ875IGcU63GuHZZbzIWg.webp)

`axes = 1`: last 1 and first 1 are both **vector**, just do vector dot product

> 由于1D数组（vector）只有一个axis，因此最后一维和第一维都是axis=0。  
> 因此，axes=1 等效于 axes=(0,0)，都沿着横向row，求点积（相同位置相乘再求和）。  

![vector-tensordot-1](https://miro.medium.com/max/1400/1*pnsDcjapYVQWLJtWe_-Dhw.webp)

In [ ]:
# 1D array

x = np.array([1,2,3])
y = np.array([4,5,6])

# The last zero axis for x is just the scalar elements of x.
# Each instance of x is a scalar, we only end up multiplying x[i] by y.
tdot0 = np.tensordot(x, y, axes=0)
print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('i,j->ij', x, y)
# print('es0:\n', es0)

# axes=1 等价于 (0,0)
# 等价于 dot,vdot,@ 等运算
tdot1 = np.tensordot(x, y, axes=1)
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('i,i', x, y)
print('es1:\n', es1)

# axes=2 等价于 ([0,1],[0,1])
# 对于一维向量超出了维度范围，将报错
# IndexError: tuple index out of range
# tdot2 = np.tensordot(x, y, axes=2)


### 1D · 2D

$ x = \begin{bmatrix} 1 & 2 & 3 \\ \end{bmatrix} $，
$ y = \begin{bmatrix} 4 & 5 & 6 \\ 5 & 6 & 7 \\ \end{bmatrix} $，

`axes = 1`: last 1 axis in a and first 1 axis in b，等价于(0,0)。

- 一维向量x的长度为3，和矩阵y的列数2模数不匹配，将报 shape-mismatch 错误。  
- 可以将axes调整为(0,1)，这样shape模数匹配，可以沿行方向求点积。  

```Python
# 结果为 [np.dot(x, y[0]), np.dot(x, y[1])]
z = np.zeros((y.shape[0],), dtype=np.int32)
for i in range(len(y)):
    z[i] = x.dot(y[i])
```

`axes = 2`：等效于 ([0,1],[0,1])，对于一维向量x，维数索引超标！

In [ ]:
# 1D · 2D

x = np.array([1, 2, 3])
y = np.array([[4, 5, 6],
             [5, 6, 7]])

tdot0 = np.tensordot(x,y,axes=0)
print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('i,jk->ijk', x, y)
# print('es0:\n', es0)

# ValueError: shape-mismatch for sum
# tdot1 = np.tensordot(x,y,axes=1)
tdot1 = np.tensordot(x,y,axes=(0,1))
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('i,ji->j', x, y)
print('es1:\n', es1)

# IndexError: tuple index out of range
# tdot2 = np.tensordot(x,y)

### 2D · 1D

$ x = \begin{bmatrix} 4 & 5 & 6 \\ 5 & 6 & 7 \\ \end{bmatrix} $，
$ y = \begin{bmatrix} 1 & 2 & 3 \\ \end{bmatrix} $，

`axes = 1`: last 1 axis in a and first 1 axis in b，等价于(1,0)。

- shape模数匹配，沿行方向求点积。

```Python
# 结果为 [np.dot(x[0], y), np.dot(x[1], y)]
z = np.zeros((x.shape[0],), dtype=np.int32)
for i in range(len(x)):
    z[i] = x[i].dot(y)
```

`axes = 2`：等效于 ([0,1],[0,1])，对于一维向量y，维数索引超标！

In [ ]:
# 2D · 1D

x = np.array([[4, 5, 6],
             [5, 6, 7]])
y = np.array([1, 2, 3])

tdot0 = np.tensordot(x,y,axes=0)
print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('ij,k->ijk', x, y)
# print('es0:\n', es0)

# ValueError: shape-mismatch for sum
# tdot1 = np.tensordot(x,y,axes=1)
tdot1 = np.tensordot(x,y,axes=1)
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('ij,j->i', x, y)
print('es1:\n', es1)

# IndexError: tuple index out of range
# tdot2 = np.tensordot(x,y)

### 2D · 2D

$ x = \begin{bmatrix} 1 & 2 & 3 \\ 2 & 3 & 4 \\ \end{bmatrix} $，
$ y = \begin{bmatrix} 4 & 5 & 6 \\ 5 & 6 & 7 \\ \end{bmatrix} $，

`axes = 0`: last 0 of x is **scalar**, and fist 0 of y is the whole matrix.

![axes=0](https://miro.medium.com/max/1392/1*aQMo5AKweWCPRY_Sh8-yjg.webp)

`axes = 1`: last 1 axis in a and first 1 axis in b, just exec **matmul**.

- 等效于 axes = (1,0) 或 [1, 0] 或 ((1),(0))，各一个方向，尝试做 row(x)·col(y)。

> take each instance of x along last one axis, so that will give us two vectors of length 3(row),  
and perform the dot product with each instance of y along first axis which is vectors of length 2(col),  
we will get a **ValueError**: shape-mismatch for sum.

将 axes 修改为 ([1,1])，相当于 $x{\cdot}y^T$，则可正常计算。

```Python
# transpose y to get it in the shape of (3,2)
tdot1 = np.tensordot(x, y.T, axes=1)
print('tdot1:\n', tdot1)
# use tensordot tuple notation to specify which axis to use
# row vector by row vector
tdot1 = np.tensordot(x,y, axes = ((1),(1)))
print('tdot1:\n', tdot1)
```

等效的计算过程如下：

```Python
# 结果为 [[np.dot(x[0], y[0]), np.dot(x[0], y[1])], [np.dot(x[1], y[0]), np.dot(x[1], y[1])]]
z=np.zeros((x.shape[0], y.shape[0]), dtype=np.int32)
for i in range(len(x)):
    for j in range(len(y)):
            z[i,j] = x[i].dot(y[j])
```

![axes=1](https://miro.medium.com/max/1400/1*zdn0X1xyxWRB1sS1PB91Fg.webp)

`axes = 2`: last 2 axis(0,1) in a and first 2 axis(0,1) in b.

- 等效于 axes = ([0, 1],[0, 1]) 或 ((1,0),(1,0))（相当于同时转置），各两个方向，相当于整个矩阵方块。
- multiply element wise, and add all the resulting element together to return a scalar.

> 两个矩阵具有相同的shape=(2,2)，可以想象将两个矩阵方块堆叠重合（overlap），然后对应位置元素相乘（multiply）再求和（sum）。  
> 此时等效于`vdot`算子，将两个矩阵flat/ravel打平再做点积求和，最终得到一个数值标量。

等价于 `np.sum(x * y)` 或 `np.vdot(x,y)`。

![axes=2](https://miro.medium.com/max/1106/1*IYSJK-3_Z1vYzYNzkDwjqg.webp)


In [ ]:
# 2D square matrix

x = np.array([1,2,3,4]).reshape(2,2)
y = np.array([5,6,7,8]).reshape(2,2)

# x的4个标量位置变成4个矩阵
# travel on x by axis 0, then axis 1, no summing is necessary
# x00=x[0,0]; x01=x[0,1]; x10=x[1,0]; x11=x[1,1]
# np.dot(x00, y), np.dot(x01, y)
# np.dot(x10, y), np.dot(x11, y)
tdot0 = np.tensordot(x, y, axes=0)
print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('ij,kl->ijkl', x, y)
# print('es0:\n', es0)

# shape-match，计算标准内积 x·y（row(a)·col(b))
# 等价于 dot,@ 等运算
tdot1 = np.tensordot(x, y, axes=1) # 等价于 axes=(1,0)
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('ij,jk->ik', x, y)
print('es1:\n', es1)

# 等价于 z=x*y, 再计算 z.sum()
# 等价于 np.dot(x.flatten(), y.flatten())
# vdot=(1,2,3,4)*(5,6,7,8)=(1,3)*(5,7)+(2,4)*(6,8)=1*5+3*7+2*6+4*8
tdot2 = np.tensordot(x, y) # axes=2 等价于 axes=([0,1],[0,1])
print('tdot2:\n', tdot2)
# 等效的einsum表达式
es2 = np.einsum('ij,ij', x, y)
print('es2:\n', es2)

In [ ]:
# 2D wide matrix

x= np.array([[1, 2, 3],
             [2, 3, 4]])
y = np.array([[4, 5, 6],
              [5, 6, 7]])

# x的每个元素值标量乘以矩阵y，放缩矩阵作为方块放到x元素位置
tdot0 = np.tensordot(x, y, axes=0)
print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('ij,kl->ijkl', x, y)
# print('es0:\n', es0)

# ValueError: shape-mismatch for sum, (row 3, col 2)
# tdot1 = np.tensordot(x, y, axes=1)

# first the -Nth axis in a and 0th axis in b: -1 for a(row) with 0 for b(col), shape-mismatch
# and the -1th axis in a and Nth axis in b last: -1 for a(row) with 1 for b(row), shape-match
# 第2种尝试情形，等效于 x·(y.T)
# np.tensordot(x, y, axes=(1,1))
tdot1 = np.tensordot(x, y.T, axes=1)
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('ij,kj->ik', x, y)
print('es1:\n', es1)

# 等价于 z=x*y, 再计算 z.sum()
# 等价于 vdot=np.dot(x.flatten(), y.flatten())
tdot2 = np.tensordot(x, y, axes=2)
print('tdot2:\n', tdot2)
# 等效的einsum表达式
es2 = np.einsum('ij,ij', x, y)
print('es2:\n', es2)

### 3D · 2D

$ x = \left[ \begin{array}{cc|cc} 1 & 2 & 3 & 4 \\ \hline 5 & 6 & 7 & 8 \end{array} \right] $ = $ \begin{bmatrix} x_{00} & x_{01} \\ x_{10} & x_{11} \\ \end{bmatrix} $，每一块 $m_i$ 代表第三维长度为2的向量。

$ y = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} $

`axes = 0`: 同上。

`axes = 1`: 等效于 axes=(2,0)，x沿第三维（纵深方向）、y沿第一维（列方向）。

- tdot1.shape=(2,2,2)。

等效计算过程：

```Python
z = np.zeros((x.shape[0], x.shape[1], x.shape[2]), dtype=np.int32)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        for k in range(y.shape[-1]):
            z[i,j,k] = x[i,j].dot(y[:,k])
```

计算过程示意：

```
np.dot(x00, y), np.dot(x01, y)
np.dot(x10, y), np.dot(x11, y)
# 更详细的计算分解
np.dot(x00, y.col0), np.dot(x00, y.col1); np.dot(x01, y.col0), np.dot(x01, y.col1);
np.dot(x10, y.col0), np.dot(x10, y.col1); np.dot(x11, y.col0), np.dot(x11, y.col1);
```

计算结果如下：

```Python
[
    [
        [np.dot(x[0,0], y[:,0]), np.dot(x[0,0], y[:,1])],
        [np.dot(x[0,1], y[:,0]), np.dot(x[0,1], y[:,1])]
    ],
    [
        [np.dot(x[1,0], y[:,0]), np.dot(x[1,0], y[:,1])],
        [np.dot(x[1,1], y[:,0]), np.dot(x[1,1], y[:,1])]
    ]
]
```

`axes = 2`: 等效于 axes=([1,2],[0,1])，x的第二、三维矩阵和y的第一、二维矩阵进行shape overlap和multiply sum。

- tdot2.shape 为 (2,)。

等效计算过程：

```Python
z = np.zeros((x.shape[0],), dtype=np.int32)
for i in range(len(x)):
    z[i] = np.vdot(x[i], y)
```

一维索引 x[0]、x[1] 为第二、三维矩阵，结果等价于 `[np.vdot(x[0],y), np.vdot(x[1],y)]`。

$ \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} $ = (1,2)(1,2)+(3,4)(3,4) = 30  
$ \begin{bmatrix} 5 & 6 \\ 7 & 8 \\ \end{bmatrix} \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} $ = (5,6)(1,2)+(7,8)(3,4) = 70  


In [ ]:
# 3D · 2D 1

x = np.arange(1, 9).reshape(2,2,2)
y = np.arange(1, 5).reshape(2,2)
# x的每个元素按位置乘以y，x的元素位置变为y阶方块
tdot0 = np.tensordot(x, y, axes=0)
# print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('ijk,lm->ijklm', x, y)
# print('es0:\n', es0)

tdot1 = np.tensordot(x, y, axes=1)
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('ijk,kl->ijl', x, y)
print('es1:\n', es1)

tdot2 = np.tensordot(x, y, axes=2)
print('tdot2:\n', tdot2)
# 等效的einsum表达式
es2 = np.einsum('ijk,jk->i', x, y)
print('es2:\n', es2)


In [ ]:
# 3D · 2D 2

# 官方例程
a = np.array(range(1, 9))
a.shape = (2, 2, 2)
A = np.array(('a', 'b', 'c', 'd'), dtype=object)
A.shape = (2, 2)

tdot0 = np.tensordot(a, A, axes=0)
# print('tdot0:\n', tdot0)
tdot1 = np.tensordot(a, A, axes=1)
print('tdot1:\n', tdot1)
tdot2 = np.tensordot(a, A)
print('tdot2:\n', tdot2)

### 2D · 3D

$ x = \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} $ = $ \begin{bmatrix} x_{00} & x_{01} \\ x_{10} & x_{11} \\ \end{bmatrix} $

$ y = \left[ \begin{array}{cc|cc} 1 & 2 & 3 & 4 \\ \hline 5 & 6 & 7 & 8 \end{array} \right] $ = $ \begin{bmatrix} y_{00} & y_{01} \\ y_{10} & y_{11} \\ \end{bmatrix} $，每一块 $m_i$ 代表第三维长度为2的向量。

`axes = 0`: 同上。

`axes = 1`: 等效于 axes=(1,0)，x沿第二维（行方向）、y沿第一维（列方向）。

- tdot1.shape=(2,2,2)。

x的行点积y的列（收缩第三维），等效计算的四个方块如下：

$ \begin{bmatrix} 1 & 2 \end{bmatrix} \begin{bmatrix} 1 & 2 \\ 5 & 6 \\ \end{bmatrix} $
$ \begin{bmatrix} 1 & 2 \end{bmatrix} \begin{bmatrix} 3 & 4 \\ 7 & 8 \\ \end{bmatrix} $  
$ \begin{bmatrix} 3 & 4 \end{bmatrix} \begin{bmatrix} 1 & 2 \\ 5 & 6 \\ \end{bmatrix} $
$ \begin{bmatrix} 3 & 4 \end{bmatrix} \begin{bmatrix} 3 & 4 \\ 7 & 8 \\ \end{bmatrix} $  

等效计算过程：

```Python
z=np.zeros((x.shape[0], y.shape[0], y.shape[-1]), dtype=np.int32)
for i in range(len(x)):
    for j in range(y.shape[1]):
      for k in range(y.shape[2]):
          z[i,j,k] = x[i].dot(y[:,j,k])
```

计算结果如下：

```Python
[
    [np.dot(x[0], y[:,0]), np.dot(x[0], y[:,1])],
    [np.dot(x[1], y[:,0]), np.dot(x[1], y[:,1])]
]
```

`axes = 2`：等效于 axes=([0,1],[0,1])，x的第一、二维矩阵和y的第一、二维矩阵进行shape overlap和multiply sum over。

- tdot2.shape 为 (2,)。

等效计算过程：

```Python
z=np.zeros((y.shape[-1]), dtype=np.int32)
for i in range(y.shape[-1]):
    z[i] = np.vdot(x, y[:,:,i])
```

计算结果如下：

- tdot2[0,0] = `np.vdot(x, y[:,:,0])` = $ x_{00}*y_{00}[0] + x_{01}*y_{01}[0] + x_{10}*y_{10}[0] + x_{11}*y_{11}[0] $  
- tdot2[0,1] = `np.vdot(x, y[:,:,1])` = $ x_{00}*y_{00}[1] + x_{01}*y_{01}[1] + x_{10}*y_{10}[1] + x_{11}*y_{11}[1] $

In [ ]:
# 2D · 3D

x = np.arange(1, 5).reshape(2,2)
y = np.arange(1, 9).reshape(2,2,2)
# x的每个元素按位置乘以y，x的元素位置变为y阶方块
tdot0 = np.tensordot(x, y, axes=0)
# print('tdot0:\n', tdot0)
# 等效的einsum表达式
es0 = np.einsum('ij,klm->ijklm', x, y)
# print('es0:\n', es0)

# 分块内积
tdot1 = np.tensordot(x, y, axes=1)
print('tdot1:\n', tdot1)
# 等效的einsum表达式
es1 = np.einsum('ij,jkl->ikl', x, y)
print('es1:\n', es1)

# 块点积相加
tdot2 = np.tensordot(x, y, axes=2)
print('tdot2:\n', tdot2)
# 等效的einsum表达式
es2 = np.einsum('ij,ijk->k', x, y)
print('es2:\n', es2)

### 3D · 3D

[numpy.tensordot](https://numpy.org/doc/stable/reference/generated/numpy.tensordot.html) 给出的官方示例：

- x = np.arange(60).reshape(3,4,5)
- y = np.arange(24).reshape(4,3,2)

`axes = 0`: 同上。

`axes = 1`: 等效于 axes=(2,0)，x沿第三维（纵深方向）、y沿第一维（列方向），分量分别为5、4，无法执行点积运算。

- ValueError: shape-mismatch for sum

`axes = 2`: 等效于 axes=([1,2],[0,1])，x的第二、三维矩阵(4x5)和y的第一、二维矩阵(4x3)进行shape overlap和multiply sum over。

- ValueError: shape-mismatch for sum

---

以下指定a按照(dim2,dim1)、b按照(dim1,dim2)，其shape都为(4,3)，符合点积运算阶数要求。

```Python
c = np.tensordot(a,b, axes=((1,0),(0,1))) # axes=([1,0],[0,1])
```

- (4,3,5)与(4,3,2) 进行tensordot最终的shape为(5,2)。  

$ x = \begin{bmatrix} x_{00} & x_{01} & x_{02} & x_{03} \\ x_{10} & x_{11} & x_{12} & x_{13} \\ x_{20} & x_{21} & x_{22} & x_{23} \\ \end{bmatrix} $，3D数组默认的axis顺序是 0,1(,2)，(1,0)相当于转置（暂不考虑第三维展开）。  
$ x^T = \begin{bmatrix} x_{00} & x_{10} & x_{20} \\ x_{01} & x_{11} & x_{21} \\ x_{02} & x_{12} & x_{22} \\ x_{03} & x_{13} & x_{23} \\ \end{bmatrix} $，
$ y = \begin{bmatrix} y_{00} & y_{01} & y_{02} \\ y_{10} & y_{11} & y_{12} \\ y_{20} & y_{21} & y_{22} \\ y_{30} & y_{31} & y_{32} \\ \end{bmatrix} $，$x^T$ 和 $y$ 的 shape 相同，都是(4,3)，支持点积。

把 $y$ 覆盖（overlap）到 $x^T$ 上，然后对应位置的块内进行运算。  
姑且把x[k,n]和y[n,k]视作2D数组的块，块内展开是第三维列表。

c[i,j] = $x^T$ 所有块的第三维 [i] 元素与 $y$ 块下相同位置的第三维 [j] 元素相乘再相加求和。

- 索引取值范围：$ i \in [0,4], j \in [0,1] $

首先，对a的第一、二维进行转置：at = `np.swapaxes(a, 0, 1)` 或 `np.transpose(a, axes=(1,0,2))`。  
然后，基于at和b重叠位置的第三维向量长度分别为5和2，两重循环计算出(5,2)的矩阵。

基于 **vdot** 及 **einsum** 的等效计算过程如下：

```Python
# 等效的vdot算法
at = np.swapaxes(a, 0, 1)
vc = np.zeros((at.shape[-1],b.shape[-1]), dtype=np.int32)
for i in range(at.shape[-1]):
    for j in range(b.shape[-1]):
        vc[i,j] = np.vdot(at[:,:,i], b[...,j])
print('vc:\n', vc)

# 等效的einsum表达式
es = np.einsum('jik,ijl->kl', a, b)
print('es:\n', es)
```

其中，部分结果元素的计算示意如下：

- c[0,0] = np.vdot(at[:,:,0], b[...,0])
- c[2,1] = np.vdot(at[:,:,2], b[...,1])
- c[4,1] = np.vdot(at[:,:,4], b[...,1])

---

官方样例中给出了等效的for循环朴素算法，也可以帮助理解其计算机制：

c[0,0] 的计算过程：

```
i, j = 0 0
a[0,0,0]=0, b[0,0,0]=0, d[0,0]=0
a[0,1,0]=5, b[1,0,0]=6, d[0,0]=30
a[0,2,0]=10, b[2,0,0]=12, d[0,0]=150
a[0,3,0]=15, b[3,0,0]=18, d[0,0]=420
a[1,0,0]=20, b[0,1,0]=2, d[0,0]=460
a[1,1,0]=25, b[1,1,0]=8, d[0,0]=660
a[1,2,0]=30, b[2,1,0]=14, d[0,0]=1080
a[1,3,0]=35, b[3,1,0]=20, d[0,0]=1780
a[2,0,0]=40, b[0,2,0]=4, d[0,0]=1940
a[2,1,0]=45, b[1,2,0]=10, d[0,0]=2390
a[2,2,0]=50, b[2,2,0]=16, d[0,0]=3190
a[2,3,0]=55, b[3,2,0]=22, d[0,0]=4400
```

c[2,1] 的计算过程：

```
i, j = 2 1
a[0,0,2]=2, b[0,0,1]=1, d[2,1]=2
a[0,1,2]=7, b[1,0,1]=7, d[2,1]=51
a[0,2,2]=12, b[2,0,1]=13, d[2,1]=207
a[0,3,2]=17, b[3,0,1]=19, d[2,1]=530
a[1,0,2]=22, b[0,1,1]=3, d[2,1]=596
a[1,1,2]=27, b[1,1,1]=9, d[2,1]=839
a[1,2,2]=32, b[2,1,1]=15, d[2,1]=1319
a[1,3,2]=37, b[3,1,1]=21, d[2,1]=2096
a[2,0,2]=42, b[0,2,1]=5, d[2,1]=2306
a[2,1,2]=47, b[1,2,1]=11, d[2,1]=2823
a[2,2,2]=52, b[2,2,1]=17, d[2,1]=3707
a[2,3,2]=57, b[3,2,1]=23, d[2,1]=5018
```

c[4,1] 的计算过程：

```
i, j = 4 1
a[0,0,4]=4, b[0,0,1]=1, d[4,1]=4
a[0,1,4]=9, b[1,0,1]=7, d[4,1]=67
a[0,2,4]=14, b[2,0,1]=13, d[4,1]=249
a[0,3,4]=19, b[3,0,1]=19, d[4,1]=610
a[1,0,4]=24, b[0,1,1]=3, d[4,1]=682
a[1,1,4]=29, b[1,1,1]=9, d[4,1]=943
a[1,2,4]=34, b[2,1,1]=15, d[4,1]=1453
a[1,3,4]=39, b[3,1,1]=21, d[4,1]=2272
a[2,0,4]=44, b[0,2,1]=5, d[4,1]=2492
a[2,1,4]=49, b[1,2,1]=11, d[4,1]=3031
a[2,2,4]=54, b[2,2,1]=17, d[4,1]=3949
a[2,3,4]=59, b[3,2,1]=23, d[4,1]=5306
```


In [ ]:
# 3D · 3D

a = np.arange(60).reshape(3,4,5)
b = np.arange(24).reshape(4,3,2)
c = np.tensordot(a,b, axes=((1,0),(0,1))) # axes=([1,0],[0,1])

# 等效的多维for循环朴素算法
d = np.zeros((5,2), dtype=np.int32)
for i in range(5):
  for j in range(2):
    print('i, j =', i, j)
    for k in range(3):
      for n in range(4):
        d[i,j] += a[k,n,i] * b[n,k,j]
        print('a[{2},{3},{0}]={4}, b[{3},{2},{1}]={5}, d[{0},{1}]={6}'.format(i,j,k,n,a[k,n,i], b[n,k,j], d[i,j]))
